## Question 3: How can I estimate the expected revenue for a new listing?

To answer the third question, I will use machine learning to predict an apartment’s potential annual revenue based on the available features (e.g., neighborhood, size, amenities, occupancy, and reviews).

## Gather the data
I am using a jupiter notebook in Cloud
Also the data was downloadied and importied on my personal S3 bucket also in cloud
I was using pandas dataframe for reading the data, sp let se how the data are looking like. 

In [ ]:
import pandas as pd

df = pd.read_csv("s3://pcp-analytics-data-cdl/Jonida/Udacity/Project1/listings.csv.gz", compression="gzip")      # infers gzip by .gz
df.head()


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,27262,https://www.airbnb.com/rooms/27262,20250926033717,2025-09-27,previous scrape,"AQA-No7, Great mattress, high speed internet",THE MATTRESS - KING KOIL - Camden Luxury 160x2...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,37177,...,4.97,4.75,4.71,00002433111,t,1,1,0,0,0.18
1,33945,https://www.airbnb.com/rooms/33945,20250926033717,2025-09-26,city scrape,Spacious Cosy aprtm very close to Metro!,Apartment located near metro station. Safe nei...,Neighbourhood is alive all day and safe all da...,https://a0.muscache.com/pictures/1a7a1026-f5f1...,146553,...,4.91,4.64,4.81,00000874421,f,1,1,0,0,0.49
2,49489,https://www.airbnb.com/rooms/49489,20250926033717,2025-09-26,city scrape,Ermou 44 - 3bdr apt in the heart of Athens,"Fully furnished, 98 sq.mt., apartment on 44 Er...",The apartment is located at the commercial cen...,https://a0.muscache.com/pictures/9640239/4bfd2...,225612,...,4.87,4.89,4.69,00000876484,t,3,3,0,0,0.83
3,60394,https://www.airbnb.com/rooms/60394,20250926033717,2025-09-26,city scrape,Cosy apartment! Great central Athens location!,Fully furnished cozy apartment in the best nei...,Very centrally located with a feeling of a qui...,https://a0.muscache.com/pictures/477282/db550f...,290864,...,4.97,4.92,4.92,00000508830,f,1,1,0,0,0.29
4,134340,https://www.airbnb.com/rooms/134340,20250926033717,2025-09-27,previous scrape,Neo Pagrati Athens Apartment,Little gem with the best of all worlds! Locate...,Our street is a small one with very little tra...,https://a0.muscache.com/pictures/1232312/4edeb...,659406,...,4.93,4.56,4.83,00002821300,f,1,1,0,0,0.36


In [2]:
#Lets see how many columns we have and what information has every column. 

print(df.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

## Prepare Data
Select the needed columns. For the third question I need to make a revenue prediction for this reason I will need the price information but also the apartment and location information.  

In [2]:
df = df[['id',
       'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds',  'price',
       'review_scores_value', 'number_of_reviews_ly', 'review_scores_location', 'availability_365']]
df.head()

,id,neighbourhood_cleansed,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365
0,27262,ΑΜΠΕΛΟΚΗΠΟΙ,Entire home/apt,2,NaN,1 bath,1.0,NaN,NaN,4.71,5,4.75,0
1,33945,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,Entire home/apt,4,1.0,1 bath,2.0,3.0,$45.00,4.81,8,4.64,269
2,49489,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Entire home/apt,7,1.5,1.5 baths,3.0,4.0,$160.00,4.69,4,4.89,72
3,60394,ΣΤΑΔΙΟ,Entire home/apt,2,1.0,1 bath,1.0,3.0,$50.00,4.92,5,4.92,54
4,134340,ΠΑΓΚΡΑΤΙ,Entire home/apt,2,NaN,1 bath,1.0,NaN,NaN,4.83,4,4.56,0


In [3]:
## As bevor I am only intersting on Apartment / Entire home 

df = df[df["room_type"] == "Entire home/apt"]

In [5]:
##Let's also examine the distribution of neighborhoods and the number of apartments in each one.
counts = df["neighbourhood_cleansed"].value_counts(dropna=False)
print(counts) 

ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ              2929
ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ                 1359
ΝΕΟΣ ΚΟΣΜΟΣ                         1134
ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ            1028
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ     864
ΠΑΓΚΡΑΤΙ                             613
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                      532
ΚΟΛΩΝΑΚΙ                             498
ΑΚΡΟΠΟΛΗ                             463
ΑΜΠΕΛΟΚΗΠΟΙ                          452
ΠΕΤΡΑΛΩΝΑ                            444
ΘΗΣΕΙΟ                               418
ΚΕΡΑΜΕΙΚΟΣ                           387
ΣΤΑΔΙΟ                               273
ΚΥΨΕΛΗ                               267
ΓΟΥΔΙ                                255
ΙΛΙΣΙΑ                               216
ΖΑΠΠΕΙΟ                              205
ΛΥΚΑΒΗΤΤΟΣ                           188
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                     158
ΝΙΡΒΑΝΑ                              144
ΠΑΤΗΣΙΑ                              144
ΚΟΛΩΝΟΣ                              127
ΑΝΩ ΚΥΨΕΛΗ                           122
ΒΟΤΑΝΙΚΟΣ       

In [5]:
### Let's also examine how many reviews we received per neighborhood over the past year.

df2 = df[df['neighbourhood_cleansed'].notna()].copy()

# group and sum
neigh_reviews = (
    df2
    .groupby('neighbourhood_cleansed', as_index=False)['number_of_reviews_ly']
    .sum()
    .rename(columns={'number_of_reviews_ly': 'total_reviews_ly'})
    .sort_values('total_reviews_ly', ascending=False)
    .reset_index(drop=True)
)

print(neigh_reviews.head(50))

              neighbourhood_cleansed  total_reviews_ly
0             ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ             40943
1                ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ             22500
2                        ΝΕΟΣ ΚΟΣΜΟΣ             16239
3   ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ             11901
4           ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ             10476
5                           ΑΚΡΟΠΟΛΗ              7717
6                             ΘΗΣΕΙΟ              7600
7                         ΚΕΡΑΜΕΙΚΟΣ              6567
8                           ΠΑΓΚΡΑΤΙ              5693
9                          ΠΕΤΡΑΛΩΝΑ              5182
10                   ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ              5088
11                       ΑΜΠΕΛΟΚΗΠΟΙ              4618
12                           ΖΑΠΠΕΙΟ              4600
13                          ΚΟΛΩΝΑΚΙ              4293
14                            ΣΤΑΔΙΟ              3404
15                           ΝΙΡΒΑΝΑ              2058
16                            ΚΥΨΕΛΗ              1858
17        

In [6]:
## We can drop the "room_type" column, since the dataset is already filtered to include only "Entire home/apt" listings.

df = df.drop(columns=[ 'room_type'])
df.head()

,id,neighbourhood_cleansed,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365
0,27262,ΑΜΠΕΛΟΚΗΠΟΙ,2,NaN,1 bath,1.0,NaN,NaN,4.71,5,4.75,0
1,33945,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,4,1.0,1 bath,2.0,3.0,$45.00,4.81,8,4.64,269
2,49489,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,7,1.5,1.5 baths,3.0,4.0,$160.00,4.69,4,4.89,72
3,60394,ΣΤΑΔΙΟ,2,1.0,1 bath,1.0,3.0,$50.00,4.92,5,4.92,54
4,134340,ΠΑΓΚΡΑΤΙ,2,NaN,1 bath,1.0,NaN,NaN,4.83,4,4.56,0


Based on the previous two analyses and the charts, I recommend focusing on these four neighborhoods, 
as they showed the strongest review scores and highest demand, making them the most promising targets for investment.

In [7]:
list = ["1Ο ΝΕΚΡΟΤΑΦΕΙΟ", "ΑΝΩ ΠΑΤΗΣΙΑ", "ΣΤΑΔΙΟ", "ΛΥΚΑΒΗΤΤΟΣ"]
df = df[df["neighbourhood_cleansed"].isin(list)]

In [11]:
df.head()

,id,neighbourhood_cleansed,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365
3,60394,ΣΤΑΔΙΟ,2,1.0,1 bath,1.0,3.0,"[""Hangers"", ""Dedicated workspace"", ""Kitchen"", ...",$50.00,4.92,5,4.92,54
27,691025,ΣΤΑΔΙΟ,4,1.0,1 bath,2.0,2.0,"[""Hangers"", ""Clothing storage: closet"", ""Kitch...",$66.00,4.82,2,4.89,261
40,807239,ΑΝΩ ΠΑΤΗΣΙΑ,2,1.0,1 bath,1.0,1.0,"[""Essentials"", ""Hangers"", ""Wifi"", ""Washer"", ""L...",$43.00,4.75,0,4.75,363
63,1272844,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,2,1.0,1 bath,1.0,1.0,"[""Hangers"", ""Kitchen"", ""Oven"", ""Refrigerator"",...",$22.00,4.62,0,4.71,237
73,1499953,ΛΥΚΑΒΗΤΤΟΣ,6,2.0,2 baths,2.0,3.0,"[""Hangers"", ""Dedicated workspace"", ""Kitchen"", ...",$156.00,4.82,13,4.92,340


In [8]:
## The bathrooms field already has many numeric values, while bathrooms_text includes written forms (e.g., "1 bath", "1.5 baths", "Half-bath", "1 1/2 baths").

## We'll extract numeric values from bathrooms_text, convert them to floats (handling commas and simple fractions), and use them to fill missing values in bathrooms.

import pandas as pd

# work on a safe copy
df = df.copy()

# normalize 'Half-bath' etc. (optional) and ensure string type
s = df['bathrooms_text'].fillna('').astype(str)
s = s.str.replace(r'(?i)\bhalf[- ]?bath\b', '0.5', regex=True)

# extract first numeric token (handles "1 bath", "1.5 baths", "0.5", etc.)
nums = pd.to_numeric(s.str.extract(r'([0-9]*\.?[0-9]+)')[0], errors='coerce')

# fill bathrooms only where it is null
df.loc[df['bathrooms'].isna(), 'bathrooms'] = nums

# ensure bathrooms is float
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').astype(float)

df.head()


,id,neighbourhood_cleansed,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365
3,60394,ΣΤΑΔΙΟ,2,1.0,1 bath,1.0,3.0,$50.00,4.92,5,4.92,54
27,691025,ΣΤΑΔΙΟ,4,1.0,1 bath,2.0,2.0,$66.00,4.82,2,4.89,261
40,807239,ΑΝΩ ΠΑΤΗΣΙΑ,2,1.0,1 bath,1.0,1.0,$43.00,4.75,0,4.75,363
63,1272844,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,2,1.0,1 bath,1.0,1.0,$22.00,4.62,0,4.71,237
73,1499953,ΛΥΚΑΒΗΤΤΟΣ,6,2.0,2 baths,2.0,3.0,$156.00,4.82,13,4.92,340


In [9]:
# drop the other not nesecary column

df = df.drop(columns=['bathrooms_text'])

In [20]:
## Have a look on the data
df.head(20)

,id,neighbourhood_cleansed,accommodates,bathrooms,bedrooms,beds,amenities,price,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365
3,60394,ΣΤΑΔΙΟ,2,1.0,1.0,3.0,"[""Hangers"", ""Dedicated workspace"", ""Kitchen"", ...",$50.00,4.92,5,4.92,54
27,691025,ΣΤΑΔΙΟ,4,1.0,2.0,2.0,"[""Hangers"", ""Clothing storage: closet"", ""Kitch...",$66.00,4.82,2,4.89,261
40,807239,ΑΝΩ ΠΑΤΗΣΙΑ,2,1.0,1.0,1.0,"[""Essentials"", ""Hangers"", ""Wifi"", ""Washer"", ""L...",$43.00,4.75,0,4.75,363
63,1272844,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,2,1.0,1.0,1.0,"[""Hangers"", ""Kitchen"", ""Oven"", ""Refrigerator"",...",$22.00,4.62,0,4.71,237
73,1499953,ΛΥΚΑΒΗΤΤΟΣ,6,2.0,2.0,3.0,"[""Hangers"", ""Dedicated workspace"", ""Kitchen"", ...",$156.00,4.82,13,4.92,340
93,2219385,ΣΤΑΔΙΟ,6,2.0,2.0,3.0,"[""Hangers"", ""Clothing storage: closet"", ""Kitch...",$65.00,4.72,5,4.74,204
101,2534822,ΛΥΚΑΒΗΤΤΟΣ,5,1.5,2.0,3.0,"[""Hangers"", ""Kitchen"", ""Oven"", ""Refrigerator"",...",$53.00,4.70,7,4.50,200
159,4399187,ΛΥΚΑΒΗΤΤΟΣ,4,1.0,1.0,1.0,"[""Hangers"", ""Kitchen"", ""Oven"", ""Refrigerator"",...",$81.00,4.84,10,4.86,82
200,5989683,ΛΥΚΑΒΗΤΤΟΣ,2,1.0,0.0,1.0,"[""Hangers"", ""Kitchen"", ""Refrigerator"", ""Pets a...",$115.00,4.55,2,4.86,360
202,6021135,ΣΤΑΔΙΟ,4,1.0,1.0,NaN,"[""Hangers"", ""Dedicated workspace"", ""Wine glass...",NaN,4.83,2,4.83,326


I would like to know the price range of the apartments and check whether the dataset contains any missing (null) values

In [4]:
counts = df["price"].value_counts(dropna=False)
print(counts) 

NaN          903
$45.00       216
$54.00       216
$60.00       202
$72.00       201
            ... 
$520.00        1
$631.00        1
$394.00        1
$1,235.00      1
$480.00        1
Name: price, Length: 582, dtype: int64


## Handeling with missing values 

As we can see, the dataset contains many missing values, so we will impute them. 

I will fill the missing entries using the mean values of comparable listings, grouped by neighborhood and accommodation type, to produce realistic estimates.




In [ ]:
#View average price by neighborhood and accommodation type
#Group the data by 'neighborhood' and 'accommodation' and compute the mean price

df1 = df.copy()

# 1) Clean price: remove leading $ and thousands separators, convert to numeric
df1['price_clean'] = (
    df1['price']
    .astype(str)
    .str.strip()
    .str.replace(r'^\$', '', regex=True)   # remove leading $
    .str.replace(r'[,\s]', '', regex=True) # remove commas/spaces
)
df1['price_clean'] = pd.to_numeric(df1['price_clean'], errors='coerce')


# 3) Optionally drop rows with missing neighbourhood or price/accommodates
df2 = df1.dropna(subset=['neighbourhood_cleansed', 'accommodates', 'price_clean']).copy()

# 4) Group and compute average price
grouped = (
    df2
    .groupby(['neighbourhood_cleansed', 'accommodates'], as_index=False)
    .agg(avg_price=('price_clean', 'mean'), n_listings=('price_clean', 'size'))
)
grouped['avg_price'] = grouped['avg_price'].round(2)
grouped = grouped.sort_values(['neighbourhood_cleansed', 'accommodates']).reset_index(drop=True)

print(grouped.head(10))

In [13]:
## We filled up the null values with the avg per group

df['price_clean'] = (
    df['price'].astype(str)
      .str.strip()
      .str.replace(r'^\$', '', regex=True)
      .str.replace(r'[,\s]', '', regex=True)
)
df['price_clean'] = pd.to_numeric(df['price_clean'], errors='coerce')

df = df.merge(grouped, on=['neighbourhood_cleansed', 'accommodates'], how='left')

# 4) fill null price_clean with the group avg
df.loc[df['price_clean'].isna(), 'price_clean'] = df.loc[df['price_clean'].isna(), 'avg_price_x']

# 5) cleanup: drop helper avg_price if you want
df = df.drop(columns=[ 'price' ,'avg_price_x', 'n_listings_x', 'avg_price_y', 'n_listings_y'])
df.head(20)

,id,neighbourhood_cleansed,accommodates,bathrooms,bedrooms,beds,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365,price_clean,avg_price,n_listings
0,60394,ΣΤΑΔΙΟ,2,1.0,1.0,3.0,4.92,5,4.92,54,50.00,64.92,64
1,691025,ΣΤΑΔΙΟ,4,1.0,2.0,2.0,4.82,2,4.89,261,66.00,127.64,84
2,807239,ΑΝΩ ΠΑΤΗΣΙΑ,2,1.0,1.0,1.0,4.75,0,4.75,363,43.00,49.80,20
3,1272844,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,2,1.0,1.0,1.0,4.62,0,4.71,237,22.00,67.91,11
4,1499953,ΛΥΚΑΒΗΤΤΟΣ,6,2.0,2.0,3.0,4.82,13,4.92,340,156.00,159.12,8
5,2219385,ΣΤΑΔΙΟ,6,2.0,2.0,3.0,4.72,5,4.74,204,65.00,108.33,18
6,2534822,ΛΥΚΑΒΗΤΤΟΣ,5,1.5,2.0,3.0,4.70,7,4.50,200,53.00,114.90,10
7,4399187,ΛΥΚΑΒΗΤΤΟΣ,4,1.0,1.0,1.0,4.84,10,4.86,82,81.00,97.62,61
8,5989683,ΛΥΚΑΒΗΤΤΟΣ,2,1.0,0.0,1.0,4.55,2,4.86,360,115.00,84.36,67
9,6021135,ΣΤΑΔΙΟ,4,1.0,1.0,NaN,4.83,2,4.83,326,127.64,127.64,84


In [27]:
# Having a look if we still have null values 
counts = df["price_clean"].value_counts(dropna=False)
print(counts) 

64.92     15
60.00     13
127.64    13
59.00     13
50.00     11
          ..
24.00      1
111.00     1
119.00     1
461.00     1
226.00     1
Name: price_clean, Length: 175, dtype: int64


In [14]:

# We have only the availability and we want to know the booking days. 
# add the column booked_days on my dataframe

df['booked_days'] = 365 - df['availability_365']

In [15]:

## ad the revenue column so we will have a overview about the apartment revenue

df['revenue_365'] = df['booked_days']*df['price_clean']


In [15]:
df.head()

,id,neighbourhood_cleansed,accommodates,bathrooms,bedrooms,beds,review_scores_value,number_of_reviews_ly,review_scores_location,availability_365,price_clean,booked_days,revenue_365
0,60394,ΣΤΑΔΙΟ,2,1.0,1.0,3.0,4.92,5,4.92,54,50.0,311,15550.0
1,691025,ΣΤΑΔΙΟ,4,1.0,2.0,2.0,4.82,2,4.89,261,66.0,104,6864.0
2,807239,ΑΝΩ ΠΑΤΗΣΙΑ,2,1.0,1.0,1.0,4.75,0,4.75,363,43.0,2,86.0
3,1272844,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,2,1.0,1.0,1.0,4.62,0,4.71,237,22.0,128,2816.0
4,1499953,ΛΥΚΑΒΗΤΤΟΣ,6,2.0,2.0,3.0,4.82,13,4.92,340,156.0,25,3900.0


In [23]:
## Having a look on the data distribution 

counts = df["neighbourhood_cleansed"].value_counts(dropna=False)
print(counts) 

ΣΤΑΔΙΟ            273
ΛΥΚΑΒΗΤΤΟΣ        188
ΑΝΩ ΠΑΤΗΣΙΑ        71
1Ο ΝΕΚΡΟΤΑΦΕΙΟ     45
Name: neighbourhood_cleansed, dtype: int64


In [16]:
# Having a look on the column that we will use for the ML algorithem 

print(df.columns)

Index(['id', 'neighbourhood_cleansed', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'review_scores_value', 'number_of_reviews_ly',
       'review_scores_location', 'availability_365', 'price_clean',
       'avg_price', 'n_listings', 'booked_days', 'revenue_365'],
      dtype='object')


## ML Model 

I use a Random Forest regressor for the revenue prediction because it handles nonlinearity and mixed feature types well. 
I split the dataset (e.g., 80% train / 20% test) to evaluate generalization on held-out data. 
Because revenue is typically right‑skewed, I apply a log1p transformation to stabilize variance and make the regression problem easier for the model.

For evaluation, I computed MAE, RMSE, and R² on the test set after back‑transforming the predictions to the original scale, 
so the metrics reflect absolute error, root-mean-square error, and explained variance in euros.

At the end I built an interactive prediction example where you can easily change all features for an apartment of interest and obtain the model’s predicted annual revenue.


In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# copy for safety
df = df.copy()

# ---- specify columns ----
numeric_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds',
                'review_scores_value', 'number_of_reviews_ly',
                'review_scores_location', 'availability_365',
                'price_clean', 'booked_days']
cat_col = 'neighbourhood_cleansed'
target_col = 'revenue_365'

# ---- ensure types & clean price_clean numeric ----
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')


# drop rows missing target
df_model = df.dropna(subset=[target_col]).copy()
X = df_model[numeric_cols + [cat_col]]
y = df_model[target_col].astype(float)

# optional: transform target to log1p (recommended if skewed)
use_log_target = True
y_model = np.log1p(y) if use_log_target else y

# ---- preprocessing ----
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler()), ])

categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='__MISSING__')),('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessor = ColumnTransformer([('num', numeric_transformer, numeric_cols),('cat', categorical_transformer, [cat_col])], remainder='drop')

# ---- model pipeline ----
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
model = Pipeline([('pre', preprocessor),('rf', rf)])

# ---- train / test split ----
X_train, X_test, y_train, y_test = train_test_split(X, y_model, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

# ---- predict & evaluate ----
y_pred = model.predict(X_test)
if use_log_target:
    y_test_orig = np.expm1(y_test)
    y_pred_orig = np.expm1(y_pred)
else:
    y_test_orig = y_test
    y_pred_orig = y_pred

print("Test MAE: ", mean_absolute_error(y_test_orig, y_pred_orig))
print("Test RMSE:", mean_squared_error(y_test_orig, y_pred_orig, squared=False))
print("Test R2:  ", r2_score(y_test_orig, y_pred_orig))

# cross-validated MAE on transformed target

cv_scores = cross_val_score(model, X, y_model, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
print("5-fold CV MAE (transformed target):", -cv_scores.mean())

# ---- helper to predict a new listing ----
def predict_revenue_for_new(model_pipeline, new_row_dict):
    """
    new_row_dict must contain numeric keys in numeric_cols and a value for 'neighbourhood_cleansed'.
    Missing numeric keys will be treated as NaN and imputed.
    Example:
      new = {'accommodates':2, 'bathrooms':1.0, 'bedrooms':1.0, 'beds':1,
             'review_scores_value':4.8, 'number_of_reviews_ly':0,
             'review_scores_location':4.9, 'availability_365':200,
             'price_clean':60.0, 'booked_days':120,
             'neighbourhood_cleansed':'ΣΤΑΔΙΟ'}
    """
    row = pd.DataFrame([new_row_dict])
    # ensure all expected columns exist
    for c in numeric_cols + [cat_col]:
        if c not in row.columns:
            row[c] = np.nan
    Xnew = row[numeric_cols + [cat_col]]
    pred = model_pipeline.predict(Xnew)[0]
    return float(np.expm1(pred)) if use_log_target else float(pred)

# example
example_new = {
    'accommodates': 2, 'bathrooms': 1.0, 'bedrooms': 1.0, 'beds': 1,
    'review_scores_value': 4.8, 'number_of_reviews_ly': 0,
    'review_scores_location': 4.9, 'availability_365': 200,
    'price_clean': 60.0, 'booked_days': 120,
    'neighbourhood_cleansed': 'ΣΤΑΔΙΟ'
}
print("Predicted revenue_365: ", predict_revenue_for_new(model, example_new))

Predicted revenue_365:  8411.35096173419

Test MAE:  1605.2658313968075
Test RMSE: 7108.064428657916
Test R2:   0.8885025220286522
5-fold CV MAE (transformed target): 0.0910434595370427
Predicted revenue_365:  8411.35096173419


In [20]:
## Hier is one more example where I use the model to predict for a new apartment with some other values

example_new = {
    'accommodates': 6, 'bathrooms': 2.0, 'bedrooms': 3.0, 'beds': 4,
    'review_scores_value': 4.8, 'number_of_reviews_ly': 0,
    'review_scores_location': 4.9, 'availability_365': 200,
    'price_clean': 100.0, 'booked_days': 200,
    'neighbourhood_cleansed': '1Ο ΝΕΚΡΟΤΑΦΕΙΟ'
}
print("Predicted revenue_365: ", predict_revenue_for_new(model, example_new))

Predicted revenue_365:  18099.989855055766
